In [ ]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2 
import datetime
import csv
import os
import math

In [ ]:
from google.colab import drive #using google drive for storage
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#declaring variables
blink_duration = []	
start_timestamp = [] 
timestamp = []  
avgBlinkDurationBuffer = []
avgBlinkFreqBuffer = []
operclos = []
average_perclos = []
eyelid_timestamp = []
eyelid_open_timestamp = []
closing_eye_movement = []
opening_eye_movement = []
headPose1 = []
kss_val = []
yawning = []
mar = []

sDir = "gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY"
EYE_AR_THRESH = 0.3
MOUTH_AR_THRESH = 0.79

In [ ]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

	
def eye_aspect_ratio(eye):
  # compute the euclidean distances between the two sets of
  # vertical eye landmarks (x, y)-coordinates

  A = dist.euclidean(eye[1], eye[5])
  B = dist.euclidean(eye[2], eye[4])
 
  # compute the euclidean distance between the horizontal
  # eye landmark (x, y)-coordinates
  C = dist.euclidean(eye[0], eye[3])
 
  # compute the eye aspect ratio
  ear = (A + B) / (2.0 * C)

  # return the eye aspect ratio
  return ear

In [ ]:
from scipy.spatial import distance as dist

(omouth, emouth) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
def mouth_aspect_ratio(mouth):
    # compute the euclidean distances between the two sets of
    # vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[2], mouth[10])  # 51, 59
    B = dist.euclidean(mouth[4], mouth[8])  # 53, 57

    # compute the euclidean distance between the horizontal
    # mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[6])  # 49, 55

    # compute the mouth aspect ratio
    mar = (A + B) / (2.0 * C)

    # return the mouth aspect ratio
    return mar

In [ ]:
image_points = np.array([
    (359, 391),     # Nose tip 34
    (399, 561),     # Chin 9
    (337, 297),     # Left eye left corner 37
    (513, 301),     # Right eye right corne 46
    (345, 465),     # Left Mouth corner 49
    (453, 469)      # Right mouth corner 55
], dtype="double")

In [ ]:
def init():

	#timestamp = [] 
	sAnnotateFile = sDir + '/annotations-auto/1-1-s2.txt'
	#opening the annotations file
	with open(sAnnotateFile, 'r') as f:
	     data = f.readlines()
	
	
	#opening timestamps file
	with open('gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY/timestamps/1-1.txt', 'r') as f1:
		time = f1.readlines()

	#opening head pose file
	#with open('gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY/headPoseValues/9-3_hp.txt', 'r') as f2:
		#headpos = f2.readlines()

	#opening kss file
	with open('gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY/kss/1-1.txt','r') as f3:
		kss = f3.readlines()

	
	FeatureExtracionDict = {'time': time, 'start_timestamp': start_timestamp, 'avgBlinkDurationBuffer': avgBlinkDurationBuffer, 'avgBlinkFreqBuffer': avgBlinkFreqBuffer, 'timestampEveryMin': timestamp, 'blink_duration': blink_duration, 'perclos': operclos, 'TimeEveryMin_eye': eyelid_timestamp, 'closingEyedur': closing_eye_movement, 'openingEyedur': opening_eye_movement, 'TimeEveryMinEyeOpen': eyelid_open_timestamp, 'average_perclos': average_perclos, 'kss': kss, 'kss_val': kss_val, "headpos1": headPose1, "MAR": mar, "Yawning": yawning}
	return FeatureExtracionDict

In [ ]:
#finding the time difference
def time_difference(start_time, end_time):
	start_time = start_time.split()	
	for i in range(0,8):
		hours = int(start_time[3])
		mins = int(start_time[4])
		secs = int(start_time[5])
		milisecs = int(start_time[6])
		microsecs = int(start_time[7])
	#converting it to microsecs
	t1, m1, s1, ms1, mis1  = hours, mins, secs, milisecs, microsecs
	start_time_microsecs = mis1 + 1000*(ms1 + 1000*(s1 + 60*(m1 + 60*t1)))
	

	end_time = end_time.split()
	for x in range(0,8,1):
		hours = int(end_time[3])
		mins = int(end_time[4])
		secs = int(end_time[5])
		milisecs = int(end_time[6])
		microsecs = int(end_time[7])	
	
	t2, m2, s2, ms2, mis2  = hours, mins, secs, milisecs, microsecs
	end_time_microsecs = mis2 + 1000*(ms2 + 1000*(s2 + 60*(m2 + 60*t2)))
	#finding the duration of blink
	time_differ = end_time_microsecs - start_time_microsecs
	#print 'time_difference in microsecs = ', time_differ
	return time_differ



#for clearing the buffer after every minute
def one_min(current_time, buffer1):
	while(len(current_time) > 0):
		
		
		ostart = current_time[0]
		oend = current_time[-1]
		diff = time_difference(ostart, oend)
		if diff > 6e+7:
			del current_time[0]
			del buffer1[0]
		else:
			break
	return current_time, buffer1	
	

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY/shape_predictor_68_face_landmarks.dat')

In [ ]:
frame_width = 1024
frame_height = 576

In [ ]:
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip 34
    (0.0, -330.0, -65.0),        # Chin 9
    (-225.0, 170.0, -135.0),     # Left eye left corner 37
    (225.0, 170.0, -135.0),      # Right eye right corne 46
    (-150.0, -150.0, -125.0),    # Left Mouth corner 49
    (150.0, -150.0, -125.0)      # Right mouth corner 55
])
def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R):
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])

def getHeadTiltAndCoords(size, image_points, frame_height):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]], [
        0, focal_length, center[1]], [0, 0, 1]], dtype="double")

    # print "Camera Matrix :\n {0}".format(camera_matrix)

    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
    (_, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points,
                                                                  camera_matrix, dist_coeffs, 
                                                                  flags = cv2.SOLVEPNP_ITERATIVE)  # flags=cv2.CV_ITERATIVE)

    # print "Rotation Vector:\n {0}".format(rotation_vector)
    # print "Translation Vector:\n {0}".format(translation_vector)
    # Project a 3D point (0, 0 , 1000.0) onto the image plane
    # We use this to draw a line sticking out of the nose_end_point2D
    (nose_end_point2D, _) = cv2.projectPoints(np.array(
        [(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

    #get rotation matrix from the rotation vector
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)

    #calculate head tilt angle in degrees
    head_tilt_degree = abs(
        [-180] - np.rad2deg([rotationMatrixToEulerAngles(rotation_matrix)[0]]))

    #calculate starting and ending points for the two lines for illustration
    starting_point = (int(image_points[0][0]), int(image_points[0][1]))
    ending_point = (int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    ending_point_alternate = (ending_point[0], frame_height // 2)

    return head_tilt_degree, starting_point, ending_point, ending_point_alternate

In [ ]:



	#capturing the video
sVideoName = sDir + "/videos_i8_crop/1-1.mp4"
cap=cv2.VideoCapture(sVideoName)
oFeatureDict = init()

landmarkPoints = []
j = 0
iFlagBlink = 0
iFlagEyelid = 0
oStartTimeBlink = 0
frame_counter = 0
while(cap.isOpened()):
		#returns the frame
	ret, frame = cap.read()
	if ret != True:
		break
	

	frame = imutils.resize(frame, width=1024, height=576)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	size = gray.shape

	# detect faces in the grayscale frame
	rects = detector(gray, 0)

	# check to see if a face was detected, and if so, draw the total
	# number of faces on the frame
	if len(rects) > 0:
			text = "{} face(s) found".format(len(rects))
			cv2.putText(frame, text, (10, 20),
									cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

	# loop over the face detections
	for rect in rects:
			# compute the bounding box of the face and draw it on the
			# frame
			(bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
			cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH), (0, 255, 0), 1)
			# determine the facial landmarks for the face region, then
			# convert the facial landmark (x, y)-coordinates to a NumPy
			# array
			shape = predictor(gray, rect)
			shape = face_utils.shape_to_np(shape)


	# coordinates = [] for training purpose 
	# landmarkPoints = oFeatureDict['data'][j].split()
	# for i in range(0,136,2):
	# 		fX = float(landmarkPoints[i])
	# 		fY = float(landmarkPoints[i + 1])
	# 		coordinates.append((fX,fY))
	# leftEye = coordinates[lStart:lEnd]
	# rightEye = coordinates[rStart:rEnd]
	
	leftEye = shape[lStart:lEnd]
	rightEye = shape[rStart:rEnd]
	mouth_shape = shape[omouth:emouth]

	mouth = mouth_aspect_ratio(mouth_shape)
	leftEAR = eye_aspect_ratio(leftEye)
	rightEAR = eye_aspect_ratio(rightEye)
	
	

	#average the eye aspect ratio together for both eyes
	ear = (leftEAR + rightEAR) / 2.0
	

	
	#check the blink count		
	if iFlagBlink == 0 and ear < EYE_AR_THRESH:	
		iFlagBlink = 1		
		oStartTimeBlink = oFeatureDict['time'][j]		
		
	if iFlagBlink==1 and ear > EYE_AR_THRESH:
		iFlagBlink=0
		end_time = oFeatureDict['time'][j]
		
		time_diff = time_difference( str(oStartTimeBlink), str(end_time)) 
		
		oFeatureDict['start_timestamp'].append(oStartTimeBlink)
		oFeatureDict['blink_duration'].append(time_diff)
		
	#calculating the average blink duration and blink frequency
	oFeatureDict['start_timestamp'], oFeatureDict['blink_duration'] = one_min(oFeatureDict['start_timestamp'],oFeatureDict['blink_duration'])
	
	if(len(oFeatureDict['blink_duration'])>0):
		avgblinkdur = sum(oFeatureDict['blink_duration'])/len(oFeatureDict['blink_duration'])
		oFeatureDict['avgBlinkDurationBuffer'].append(avgblinkdur)
		# print ('avgblinkdur', avgblinkdur)
		#print oFeatureDict['avgBlinkDurationBuffer']
	else:
		avgblinkdur = 0
		oFeatureDict['avgBlinkDurationBuffer'].append(avgblinkdur)
		# print ('avgblinkdur', avgblinkdur)

	#blink frequency
	avgBlinkFreq = len(blink_duration)
	oFeatureDict['avgBlinkFreqBuffer'].append(avgBlinkFreq)
	# print ('blinkfreq', avgBlinkFreq)
	#print oFeatureDict['avgBlinkFreqBuffer']

	
	#calculating the Perclos value
	if ear < 0.3:
		c_counter = 1
		oFeatureDict['perclos'].append(c_counter)
	else: 
		o_counter = 0
		oFeatureDict['perclos'].append(o_counter)

	oFeatureDict['timestampEveryMin'].append(oFeatureDict['time'][j])
		
	oFeatureDict['timestampEveryMin'], oFeatureDict['perclos'] = one_min(oFeatureDict['timestampEveryMin'], oFeatureDict['perclos'])

	if (len(oFeatureDict['perclos'])>0):
		avg_perclos = sum(oFeatureDict['perclos'])/float(len(oFeatureDict['perclos']))
		oFeatureDict['average_perclos'].append(avg_perclos)
		# print ('avg perclos', avg_perclos)
		#print 'average perclos', oFeatureDict['average_perclos']
	else:
		avg_perclos = 0
		oFeatureDict['average_perclos'].append(avg_perclos)
		# print ('avg perclos', avg_perclos)


	#getting the kss values
	KSS = oFeatureDict['kss'][0].rstrip('\n')
	# print (KSS)
	oFeatureDict['kss_val'].append(KSS)
	#print oFeatureDict['kss_val']
	
	
	
	#displaying the average blink duration        
	cv2.putText(frame, 'avgBlinkDur'+str(oFeatureDict['avgBlinkDurationBuffer'][j]), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)     
							#displaying levels of drowsiness
	cv2.putText(frame, 'avgperclos'+str(oFeatureDict['average_perclos'][j]), (100, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

	#displaying the blink freq
	cv2.putText(frame, 'avgblinkfreq'+str(oFeatureDict['avgBlinkFreqBuffer'][j]), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
	
	cv2.putText(frame, 'frame number'+str(j), (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

	
	# compute the convex hull for the mouth, then
	# visualize the mouth

	# Draw text if mouth is open
	mouth = shape[omouth:emouth]

	mouthMAR = mouth_aspect_ratio(mouth)
	mar = mouthMAR
	# compute the convex hull for the mouth, then
	# visualize the mouth
	mouthHull = cv2.convexHull(mouth)
 

	cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
	cv2.putText(frame, "MAR: {:.2f}".format(mar), (650, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
	oFeatureDict["MAR"].append(round(mar,2))

	# Draw text if mouth is open
	if mar > MOUTH_AR_THRESH:
			oFeatureDict["Yawning"].append(1) 
			cv2.putText(frame, "Yawning!", (20, 80),
									cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

	else:
			oFeatureDict["Yawning"].append(0)
			cv2.putText(frame, "NOT Yawning!", (20, 80),
									cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
	#showing the image
	#cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
	# cv2.imshow('frame', frame)
	for (i, (x, y)) in enumerate(shape):

		if i == 33:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[0] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		elif i == 8:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[1] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		elif i == 36:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[2] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		elif i == 45:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[3] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		elif i == 48:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[4] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		elif i == 54:
				# something to our key landmarks
				# save to our new key point list
				# i.e. keypoints = [(i,(x,y))]
				image_points[5] = np.array([x, y], dtype='double')
				# write on frame in Green
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
		else:
				# everything to all other landmarks
				# write on frame in Red
				cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
				cv2.putText(frame, str(i + 1), (x - 10, y - 10),
										cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

			#Draw the determinant image points onto the person's face
	for p in image_points:
			cv2.circle(frame, (int(p[0]), int(p[1])), 3, (0, 0, 255), -1)

	(head_tilt_degree, start_point, end_point, 
			end_point_alt) = getHeadTiltAndCoords(size, image_points, frame_height)

	cv2.line(frame, start_point, end_point, (255, 0, 0), 2)
	cv2.line(frame, start_point, end_point_alt, (0, 0, 255), 2)
	oFeatureDict['headpos1'].append(head_tilt_degree[0])

	if head_tilt_degree:
			cv2.putText(frame, 'Head Tilt Degree: ' + str(head_tilt_degree[0]), (170, 20),
									cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
	from google.colab.patches import cv2_imshow 
	cv2_imshow(frame)

	# print ('j', j)
	#print 'ear', ear
	
	if cv2.waitKey(0) & 0xff==ord('q'): 
		break
	
	j = j + 1


#releasing the cap
cap.release()
cv2.destroyAllWindows() 
	
	
	



In [ ]:
# writing the csv files for avg blink duration, blink frequency and perclos
with open('gdrive/My Drive/DROZY.zip (Unzipped Files)/DROZY/csvfile_new/1-1.csv', 'w') as f2:
	spamwriter = csv.writer(f2, delimiter=',',
													quotechar='|', quoting=csv.QUOTE_MINIMAL)

	data = list(zip(oFeatureDict['avgBlinkDurationBuffer'], oFeatureDict['avgBlinkFreqBuffer'], oFeatureDict['average_perclos'], oFeatureDict["MAR"], oFeatureDict["Yawning"], oFeatureDict["headpos1"], oFeatureDict['kss_val']))
	for row in data:
			row = list(row)
			spamwriter.writerow(row)	

In [ ]:
oFeatureDict.keys()